# ロバストMDPと正則化

参考
* [Twice regularized MDPs and the equivalence between robustness and regularization](https://arxiv.org/abs/2110.06267)
* [Distributionally Robust Optimization: A Review](https://arxiv.org/abs/1908.05659): 読もう

[ロバストMDP](RL_robust_MDP.ipynb)とエントロピー正則化には実は密接な関係があります。
実際、この関係は強化学習以外の分野におけるロバスト最適化問題では有名みたいですね ([Distributionally Robust Optimization: A Review](https://arxiv.org/abs/1908.05659)を読もう)。
今回は強化学習におけるロバスト性と正則化がほぼ等価であることを見ていきます。

**強化学習の表記**

* $\mathcal{X}:= \mathcal{S}\times \mathcal{A}$
* $\mu_0$：初期分布
* 期待リターン：$\rho(\pi):=\mathbb{E}\left[\sum_{t=0}^{\infty} \gamma^t r\left(s_t, a_t\right) \mid \mu_0, \pi, P\right]=\left\langle v_{(P, r)}^\pi, \mu_0\right\rangle$
* ベルマン期待作用素：$T_{(P, r)}^\pi v:=r^\pi+\gamma P^\pi v \quad \forall v \in \mathbb{R}^{\mathcal{S}}, \pi \in \Delta_{\mathcal{A}}^{\mathcal{S}}$,
* ベルマン最適作用素：$T_{(P, r)} v:=\max _{\pi \in \Delta_{\mathcal{A}}^{\mathcal{S}}} T_{(P, r)}^\pi v \quad \forall v \in \mathbb{R}^{\mathcal{S}}$
* 貪欲方策：$\mathcal{G}_{(P, r)}(v):=\left\{\pi \in \Delta_{\mathcal{A}}^{\mathcal{S}}: T_{(P, r)}^\pi v=T_{(P, r)} v\right\} \quad \forall v \in \mathbb{R}^{\mathcal{S}}$
* その他：
    * $r^\pi:=\left[\left\langle\pi_s, r(s, \cdot)\right\rangle\right]_{s \in \mathcal{S}}$ 
    * $P^\pi=\left[P^\pi\left(s^{\prime} \mid s\right)\right]_{s^{\prime}, s \in \mathcal{S}}$ 
    * $P^\pi\left(s^{\prime} \mid s\right):=\left\langle\pi_s, P\left(s^{\prime} \mid s, \cdot\right)\right\rangle$

[RL_as_LP.ipynb](RL_as_LP.ipynb)で詳しく解説しましたが、プランニング問題は線形計画問題に変形することができます。
これを使うと、期待リターンを次のように書き換えることができます。

* 主問題：$\rho(\pi) = \min _{v \in \mathbb{R}^{\mathcal{S}}}\left\langle v, \mu_0\right\rangle \text { subject to (s.t.) } v \geq r^\pi+\gamma P^\pi v$
* 双対問題：$\rho(\pi) = \max _{\mu \in \mathbb{R}^{\mathcal{S}}}\left\langle r^\pi, \mu\right\rangle$ s. t. $\mu \geq 0$ and $\left(\mathbf{I d}_{\mathbb{R}^{\mathcal{S}}}-\gamma P_*^\pi\right) \mu=\mu_0$

ここで、$\mathbf{I d}_{\mathbb{R}^{\mathcal{S}}}$は$\mathcal{S}$上の恒等関数、$P_*^\pi$は$P^\pi$の随伴作用素です。つまり、$\left\langle P^\pi v, v^{\prime}\right\rangle=\left\langle v, P_*^\pi v^{\prime}\right\rangle \quad \forall v, v^{\prime} \in \mathbb{R}^{\mathcal{S}}$とします。

この主問題と双対問題の解はそれぞれ価値関数と割引訪問頻度に対応します。詳しくは[RL_as_LP.ipynb](RL_as_LP.ipynb)を見てください。

**正則化**

つづいて、正則化されたMDPを考えていきます。

* $s \in \mathcal{S}, \Omega_s: \Delta_{\mathcal{A}} \rightarrow \mathbb{R}$：強凸関数
* 正則化されたベルマン期待作用素：$\left[T_{(P, r)}^{\pi, \Omega} v\right](s):=T_{(P, r)}^\pi v(s)-\Omega_s\left(\pi_s\right) \quad \forall v \in \mathbb{R}^{\mathcal{S}}, s \in \mathcal{S}$
* 正則化されたベルマン最適作用素：$T_{(P, r)}^{*, \Omega} v:=\max _{\pi \in \Delta_{\mathcal{A}}^{\mathcal{S}}}{T_{(P, r)}^{\pi, \Omega}} v \quad \forall v \in \mathbb{R}^{\mathcal{S}}$
* 正則化された価値関数（$T_{(P, r)}^{\pi, \Omega}$の不動点）：$v_{(P, r)}^{\pi, \Omega}=\left(\mathbf{I}_{\mathcal{S}}-\gamma P^\pi\right)^{-1}\left(r^\pi-\Omega(\pi)\right)$
    * これは$\tilde{r}^\pi:=r^\pi-\Omega(\pi)$のときの通常のMDPと同じ。$r^\pi$がいじられていることに注意。$r$ではないよ。

**ロバストMDP**

最後にロバストMDPの表記です。今回は報酬関数もロバストな場合を考えましょう。

* 不確実な遷移確率：$P \in \mathcal{P} \subseteq \Delta_{\mathcal{S}}^{\mathcal{X}}$
* 不確実な報酬関数：$r \in \mathcal{R} \subseteq \mathbb{R}^{\mathcal{X}}$
* 不確実集合：$\mathcal{U}:=\mathcal{P} \times \mathcal{R}$
    * ロバストMDPでは$(P, r) \in \mathcal{U}$の最悪ケースを考えます。
    * 一般の$\mathcal{U}$を考えると解けないことが知られています。そこで状態や状態行動についての直積集合$\mathcal{U}=\times_{s \in \mathcal{S}} \mathcal{U}_s=\times_{s \in \mathcal{S}}\left(\mathcal{P}_s \times \mathcal{R}_s\right)$がよく使われます。
* ロバスト期待ベルマン作用素：${\left[T^{\pi, \mathcal{U}} v\right](s):=\min _{(P, r) \in \mathcal{U}} T_{(P, r)}^\pi v(s) \quad \forall v \in \mathbb{R}^{\mathcal{S}}, s \in \mathcal{S}, \pi \in \Delta_{\mathcal{S}}^{\mathcal{A}},}$
* ロバスト最適ベルマン作用素：${\left[T^{*, \mathcal{U}} v\right](s):=\max _{\pi \in \Delta_{\mathcal{A}}^{\mathcal{S}}}\left[T^{\pi, \mathcal{U}} v\right](s) \quad \forall v \in \mathbb{R}^{\mathcal{S}}, s \in \mathcal{S},}$
* ロバストMDPでの線型計画法による最適価値関数：$\max _{v \in \mathbb{R}^{\mathcal{S}}}\left\langle v, \mu_0\right\rangle$ s.t. $v \leq T_{(P, r)}^\pi v$ for all $(P, r) \in \mathcal{U}$.
<!-- * ロバストQ関数：$q(s, a)=\min _{(P, r) \in \mathcal{U}}\{r(s, a)+\gamma\langle P(\cdot \mid s, a), v\rangle\} \quad \forall(s, a) \in \mathcal{X}$ -->

## Fenchel-Rockafellar duality

今回の話にはFenchel-Rockafellar dualityが非常に重要になります. 
$X, Y$ をユークリッド空間, $f: X \rightarrow \overline{\mathbb{R}}$ と$g: Y \rightarrow \overline{\mathbb{R}}$ をプロパーな凸関数, $A: X \rightarrow Y$ を$0 \in \operatorname{core}(\operatorname{dom}(g)-$ $A(\operatorname{dom}(f)))$であるような線形写像とします。 
このとき、

$$
\min _{x \in X} f(x)+g(A x)=\max _{y \in Y}-f^*\left(-A^* y\right)-g^*(y) .
$$

が成り立ちます。



これから先は、真のモデルの周りに不確かなMDPが広がってる状況を考えます。つまり、
$$
\left(P_0+\mathcal{P}\right) \times\left(r_0+\mathcal{R}\right)
$$
なる不確実集合を考えていきます。　

## Reward-robust MDPs

まずは報酬関数が不確かな時を考えます。つまり、
$$
\left\{P_0\right\} \times\left(r_0+\mathcal{R}\right)
$$
を想定します。
また、集合$\mathfrak{Z}$に対して$\sigma_{\mathfrak{Z}}(\mathbf{y})=\max _{\mathbf{a} \in \mathfrak{Z}}\langle\mathbf{a}, \mathbf{y}\rangle$と表記します。
ここで、次の定理が成立します。

---

**Reward-robust MDP**

$$
\max _{v \in \mathbb{R}^{\mathcal{S}}}\left\langle v, \mu_0\right\rangle s . t . v(s) \leq T_{\left(P_0, r_0\right)}^\pi v(s)-\sigma_{\mathcal{R}_s}\left(-\pi_s\right) \text { for all } s \in \mathcal{S} .
$$

の解はロバスト価値関数$v^{\pi, \mathcal{U}}$になる。

これはつまりRegularized RLにおいて、正則化関数$\Omega_s\left(\pi_s\right)$を適切に調整すると、ロバストRLと同じになることを表しています。
例えば$\mathcal{R}_s:=\left\{r_s \in \mathbb{R}^{\mathcal{A}}:\left\|r_s\right\| \leq \alpha_s^r\right\}$であれば、$\Omega_s\left(\pi_s\right):=\alpha_s^r\left\|\pi_s\right\|$の場合と同じになります。
証明してみましょう。

---

## General-robust MDPs

---

**General-robust MDP**

$\max _{v \in \mathbb{R}^{\mathcal{S}}}\left\langle v, \mu_0\right\rangle$ s. t. $v(s) \leq T_{\left(P_0, r_0\right)}^\pi v(s)-\sigma_{\mathcal{R}_s}\left(-\pi_s\right)-\sigma_{\mathcal{P}_s}\left(-\gamma v \cdot \pi_s\right)$ for all $s \in \mathcal{S}$
の解はロバスト価値関数$v^{\pi, \mathcal{U}}$になる。

このように、報酬と遷移確率の両方を考えても大丈夫です。　

TODO: この論文、大事な話ですがかなり表記がダルいですね...
もっとキレイにかけるやつ探してそっちをまとめようかな